In [1]:
import os


import instaloader
import yadisk

from decouple import config
from tqdm.notebook import tqdm
from tqdm import trange

In [2]:
L = instaloader.Instaloader(save_metadata=False, 
                            download_comments=False, 
                            download_geotags=False, 
                            download_videos=False, 
                            filename_pattern="{date_utc}_UTC_profile_{profile}",
                            quiet=True)

L.login(config('U'), config('P'))
hashtag = instaloader.Hashtag.from_name(L.context, 'tattoosketch')

In [3]:
API_KEY = config('KEY')
y = yadisk.YaDisk(token=API_KEY)

assert y.check_token()

`profile.get_posts()` is a NodeIterator instance and supports state freezing. `hashtag.get_posts()` is a python generator and does not support freezing. :(

In [31]:
def save_photos_batch(post_iterable, n_photos, save_dir_name):
    for i, post in tqdm(enumerate(post_iterable), 
                        desc='instagram images scrapping bar', 
                        total=n_photos,
                        leave=False):
        if i >= n_photos:
            break
        try:
            L.download_post(post, target=save_dir_name)
        except instaloader.B:

In [32]:
def upload_files_from_dir(src_path, dst_path, y):
    try:
        y.mkdir(dst_path)
    except yadisk.exceptions.DirectoryExistsError:
        pass
    for filename in tqdm(os.listdir(src_path), desc='uploading images to y.disk', leave=False):
        try:
            y.upload(os.path.join(src_path, filename), 
                     os.path.join(dst_path, filename))
        except yadisk.exceptions.PathExistsError:
            pass

In [33]:
def clear_dir(dir_path):
    for filename in os.listdir(dir_path):
        os.remove(os.path.join(dir_path, filename))

In [34]:
hashtag = instaloader.Hashtag.from_name(L.context, 'sketchtattoo')

In [35]:
hashtag_iterable = hashtag.get_all_posts()

In [36]:
dir_name = 'sketchtattoo_all'

for _ in tqdm(range(1000)):
    save_photos_batch(hashtag_iterable, 20, dir_name)
    upload_files_from_dir(dir_name, 'app:/insta/' + dir_name, y)
    clear_dir(dir_name)

BadResponseException: Fetching Post metadata failed.